# Import necessary modules

In [1]:
# Import necessary modules
import os
from dotenv import load_dotenv
from autogen import ConversableAgent, AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager
from autogen.tools.experimental import PerplexitySearchTool
from autogen.llm_config import LLMConfig
#from autogen import ConversableAgent, LLMConfig

import json
import time
import random

# Load environment variables
load_dotenv()

print("AG2 modules imported successfully!")

AG2 modules imported successfully!


In [2]:
import logging

# Suppress API key format warning
logging.getLogger("autogen.oai.client").setLevel(logging.ERROR)

### LLM Configurations

In [3]:
config_list=[
    {
        "model": "sonar-pro",
        "api_key": os.getenv("PERPLEXITY_API_KEY"),
        "base_url": "https://api.perplexity.ai",
        "api_type": "openai",
        "temperature": 0.3,
        "max_tokens": 1000},
    {
        "model": "llama3.2:latest",
        "api_type": 'ollama',
        "client_host": "http://192.168.0.1:11434",
        "temperature": 0.0,
        "max_tokens": 200}]

#### PERPLEXITY

In [4]:
perplexity_config = LLMConfig(config_list[0])
perplexity_config

LLMConfig(config_list=[{'max_tokens': 1000, 'temperature': 0.3, 'api_type': 'openai', 'model': 'sonar-pro', 'api_key': '**********', 'base_url': 'https://api.perplexity.ai/', 'tags': [], 'stream': False}])

#### OLLAMA

In [5]:
ollama_config = LLMConfig(config_list[1])
ollama_config

LLMConfig(config_list=[{'max_tokens': 200, 'temperature': 0.0, 'api_type': 'ollama', 'model': 'llama3.2:latest', 'tags': [], 'client_host': HttpUrl('http://192.168.0.1:11434/'), 'stream': False, 'num_predict': -1, 'num_ctx': 2048, 'repeat_penalty': 1.1, 'seed': 0, 'top_k': 40, 'hide_tools': 'never', 'native_tool_calls': False}])

## Create our LLM agent

### Student

In [6]:
student = ConversableAgent(
    name="student",
    llm_config=perplexity_config,
    system_message="You are a curious student. You ask clear, specific questions to learn new concepts.",
    human_input_mode="NEVER" 
)

#### Run the agent with a prompt

In [7]:
question = "What is AI?"
student_response = student.run(
    message=f"Answer Precisely /n {question}",
    max_turns=1,
)

# Iterate through the chat automatically with console output
student_response.process()

user (to student):

Answer Precisely /n What is AI?

--------------------------------------------------------------------------------


AuthenticationError: <html>
<head><title>401 Authorization Required</title></head>
<body>
<center><h1>401 Authorization Required</h1></center>
<hr><center>openresty/1.27.4</center>
<script>(function(){function c(){var b=a.contentDocument||a.contentWindow.document;if(b){var d=b.createElement('script');d.innerHTML="window.__CF$cv$params={r:'9b8bb97289e4ecc5',t:'MTc2NzU0MDA3My4wMDAwMDA='};var a=document.createElement('script');a.nonce='';a.src='/cdn-cgi/challenge-platform/scripts/jsd/main.js';document.getElementsByTagName('head')[0].appendChild(a);";b.getElementsByTagName('head')[0].appendChild(d)}}if(document.body){var a=document.createElement('iframe');a.height=1;a.width=1;a.style.position='absolute';a.style.top=0;a.style.left=0;a.style.border='none';a.style.visibility='hidden';document.body.appendChild(a);if('loading'!==document.readyState)c();else if(window.addEventListener)document.addEventListener('DOMContentLoaded',c);else{var e=document.onreadystatechange||function(){};docume

In [ ]:
# Response
student_response.messages

### Tutor

In [ ]:
tutor = ConversableAgent(
    name="tutor",
    system_message="You are a helpful tutor who provides clear and concise explanations suitable for a beginner.",
    human_input_mode="NEVER",
    code_execution_config=False,
    llm_config=perplexity_config  # Same config
)

In [ ]:
response = tutor.run(
    message=f"{question}"+student_response.messages[1]['content'],
    max_turns=1,
)

# Iterate through the chat automatically with console output
response.process()

### Start a 2-turn conversation initiated by the student

In [ ]:
chat_result = student.initiate_chat(
    recipient=tutor,                                # who the student is talking to
    message="Can you explain what a neural network is?",  # the student's question
    max_turns=1,                                     # total number of back-and-forth messages
    summary_method="reflection_with_llm"            # generate a final summary using LLM
)

In [ ]:
# Step 4: Print the summary of the conversation
print("\nFinal Summary:")
print(chat_result.summary)

In [ ]:
llm_config = perplexity_config
# Create a Technical Expert Agent
tech_expert = ConversableAgent(
    name="tech_expert",
    system_message="""You are a senior software engineer with expertise in Python, AI, and system design.
    Provide technical, detailed explanations with code examples when appropriate.
    Always consider best practices and performance implications.""",
    llm_config=llm_config,
    human_input_mode="NEVER"
)

# Create a Creative Writer Agent
creative_writer = ConversableAgent(
    name="creative_writer",
    system_message="""You are a creative writer and storyteller.
    Your responses are engaging, imaginative, and use vivid descriptions.
    You excel at making complex topics accessible through stories and analogies.""",
    llm_config=llm_config,
    human_input_mode="NEVER"
)

# Create a Business Analyst Agent
business_analyst = ConversableAgent(
    name="business_analyst",
    system_message="""You are a business analyst focused on ROI, efficiency, and strategic planning.
    Always consider business impact, costs, and practical implementation.
    Provide actionable recommendations with clear metrics.""",
    llm_config=llm_config,
    human_input_mode="NEVER"
)

agents = [tech_expert, creative_writer, business_analyst]
print("Specialized agents created!")
for agent in agents:
    print(f"- {agent.name}: {agent.system_message.split('.')[0]}.")

## Tools and Extensions<a name="tools"></a>

Tools extend agent capabilities beyond text conversations, allowing them to:
- Execute code
- Access external APIs
- Perform calculations
- Interact with databases
- Generate visualizations

Agents gain significant utility through tools as they provide access to external data, APIs, and functionality.  

Let's explore how to integrate tools with AG2 agents:


### What is `register_function(...)` ?

In AG2, `register_function(...)` is used to expose a Python function as a **tool** that can be executed by one agent on behalf of another. This enables agents to delegate tasks like computation, data processing, or external API calls.

#### Purpose
- Extend agent capabilities beyond text generation.
- Allow agents to solve tasks through **function execution**.
- Enable collaborative workflows between a **caller** and an **executor** agent.

#### Parameters
- **function**: A regular Python function to be used as a tool.
- **caller**: The agent that will request the tool to be used.
- **executor**: The agent that will actually execute the function.
- **description** *(optional)*: A natural language description of the function for the LLM to decide when to use it.

#### Example
```python
register_function(
    is_prime,
    caller=math_asker,
    executor=math_checker,
    description="Check if a number is prime. Returns Yes or No."
)


## Structured Outputs<a name="structured-outputs"></a>

Structured outputs ensure consistent, validated agent responses using Pydantic models.   

This is crucial for:
  
- **Data validation**: Ensuring response format consistency
- **API integration**: Reliable data exchange
- **Quality assurance**: Preventing malformed outputs
- **Type safety**: Clear data contracts

**Analogy:** Like standardized hospital forms ensure doctors always record patient info the same way, structured outputs make sure agents respond in predictable, machine-readable formats.


In AG2, structured outputs are implemented using Pydantic models and the `response_format` parameter in the `LLMConfig`.

To ensure that your agent always returns outputs in a consistent structure, you define a Pydantic class (for example, `ResponseModel`) and assign it to the `response_format` argument of your configuration.

This tells the underlying LLM to return a JSON-compatible response matching the defined schema.

```python
class ResponseModel(BaseModel):
    name: str
    status: str

llm_config = LLMConfig(
    api_type="openai",
    model="gpt-4o-mini",
    response_format=ResponseModel
)
```

With this setup:

- The agent automatically formats its responses to match the ResponseModel. 
- You don’t need to prompt the LLM to format its response.
- The response is parsed and validated by AG2.

This approach is essential for reliable automation, integrations, and downstream processing.

Let's implement structured outputs with AG2:


In [ ]:
from pydantic import BaseModel
from autogen import ConversableAgent, LLMConfig

# Define the structure of the agent's output
class TicketSummary(BaseModel):
    customer_name: str
    issue_type: str
    urgency_level: str
    recommended_action: str

llm_config = LLMConfig(
    config_list[0],
    response_format=TicketSummary,
)

# Create the agent
with llm_config:
    support_agent = ConversableAgent(
        name="support_agent",
        system_message=(
            "You are a support assistant. Summarize a customer ticket using:"
            "\n- customer_name"
            "\n- issue_type (e.g. login issue, billing problem, bug report)"
            "\n- urgency_level (Low, Medium, High)"
            "\n- recommended_action"
        ),
    )

# Start a structured conversation
support_agent.initiate_chat(
    recipient=support_agent,
    message="Ticket: John Doe is unable to reset his password and has an important meeting in 30 minutes.",
    max_turns=1
)

In [ ]:
from perplexity import Perplexity

client = Perplexity()

search = client.search.create(
    query=[
      "What is Comet Browser?",
      "Perplexity AI",
      "Perplexity Changelog"
    ]
)

for result in search.results:
    print(f"{result.title}: {result.url}")

In [ ]:
perplexity_search_tool = PerplexitySearchTool(
    api_key=os.getenv("PERPLEXITY_API_KEY"),
    max_tokens=1000
)

# Register the tool for LLM recommendation and execution.
# perplexity_search_tool.register_for_llm(assistant)
# perplexity_search_tool.register_for_execution(user_proxy)